# Pooling (Section 7.4)
1. Implement average pooling through a convolution.
2. Max-pooling can be accomplished using ReLU operations, i.e., ReLU(x) = max(0, x).

    (a) Express max(a, b) by using only ReLU operations.

    (b) Use this to implement max-pooling by means of convolutions and ReLU layers.

    (c) How many channels and layers do you need for a 2 × 2 convolution? How many
    for a 3 × 3 convolution?

3. What is the computational cost of the pooling layer? Assume that the input to the
pooling layer is of size c × h × w, the pooling window has a shape of ph × pw with a
padding of (ph, pw) and a stride of (sh, sw).

In [5]:
#!pip install d2l==1.0.3

# d2l importing
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

# libraries needed
import torch
from torch import nn
import d2l
from d2l import torch as d2l

import random

Mounted at /content/gdrive


# 1 Avg Pooling

In [3]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

pool2d(X, (2, 2), 'max')

tensor([[4., 5.],
        [7., 8.]])

## Using Conv2d function

In [4]:
import torch
import torch.nn.functional as F

def average_pooling_conv(input_tensor, kernel_size):
    # Define the kernel for average pooling
    kernel = torch.ones((1, 1, kernel_size, kernel_size)) / (kernel_size * kernel_size)

    # Perform convolution with the defined kernel
    pooled_tensor = F.conv2d(input_tensor, kernel, stride=kernel_size, padding=0)

    return pooled_tensor

# Example usage
input_tensor = torch.randn(1, 1, 4, 4)  # Example input tensor of shape (batch_size, channels, height, width)
kernel_size = 2  # Example kernel size for average pooling

# Apply average pooling through convolution
pooled_tensor = average_pooling_conv(input_tensor, kernel_size)

print("Input Tensor:")
print(input_tensor)
print("Pooled Tensor (Average Pooling):")
print(pooled_tensor)


Input Tensor:
tensor([[[[ 0.1762,  1.0988, -1.3360,  0.0105],
          [-0.1036,  0.4038, -0.9177, -0.3561],
          [-0.0431, -1.0564, -0.3834, -1.4504],
          [ 1.2262,  0.6320,  0.2845,  1.5176]]]])
Pooled Tensor (Average Pooling):
tensor([[[[ 0.3938, -0.6498],
          [ 0.1897, -0.0079]]]])


# 2 Max Pooling

## Part a
Express max(a, b) by using only ReLU operations.


In [20]:
def relu(x):
    return max(x,0)

a = random.randint(-100, 100)
b = random.randint(-100, 100)

print("a =", a, "b =", b)
print("max(a,b) = " ,max(a,b))
print("Does relu(a-b)+b == max(a,b): ", relu(a-b)+b == max(a,b))

a = -54 b = -31
max(a,b) =  -31
Does relu(a-b)+b == max(a,b):  False


## Part b
Use this to implement max-pooling by means of convolutions and ReLU layers.

In [43]:
from torch.nn import functional as F

def max_pooling_conv(input_tensor, pool_size):
    batch_size, channels, height, width = input_tensor.size()
    kernel_size = (pool_size, pool_size)
    stride = (pool_size, pool_size)
    output_shape = F.conv2d(input_tensor, torch.ones((channels, 1, pool_size, pool_size)), stride=stride, padding=0, groups=channels).shape
    output_tensor = torch.tensor([-torch.inf]*output_shape.numel()).reshape(output_shape)
    # Define the average pooling kernel
    for i in range(pool_size*pool_size):
        kernel = torch.zeros(pool_size*pool_size)
        kernel[i] = 1
        kernel = kernel.reshape(channels, 1, pool_size, pool_size)
    # Apply the convolution operation with average pooling kernel
        temp = F.conv2d(input_tensor, kernel, stride=stride, padding=0, groups=channels)
        output_tensor = relu(output_tensor - temp) + temp
    return output_tensor

# Example usage
input_tensor = torch.randn(1, 1, 6, 6)  # Batch size of 1, 3 channels, 6x6 input

pool_size = 2
output_tensor = max_pooling_conv(input_tensor, pool_size)
print(input_tensor)
print(output_tensor)

tensor([[[[ 1.5207, -0.6636,  0.3864,  0.2526, -1.1528, -0.6818],
          [-0.4233,  2.1924,  0.3049,  0.4526,  0.4283, -0.5279],
          [-0.6236, -1.3573, -1.0361,  1.1790, -0.1641, -0.9898],
          [ 1.0136,  0.8877,  0.2949,  0.6458,  0.6329, -0.3318],
          [-0.5904,  1.0632,  0.5092,  0.4589, -0.7454, -0.1808],
          [-0.6574,  0.8899, -0.1344,  0.7232, -1.0532, -1.3300]]]])
tensor([[[[ 2.1924,  0.4526,  0.4283],
          [ 1.0136,  1.1790,  0.6329],
          [ 1.0632,  0.7232, -0.1808]]]])


## Part c
How many channels and layers do you need for a 2 × 2 convolution? How many for a 3 × 3 convolution?


There is no specific requirement for each, but for an example in an RGB image there would be 3 channels needed.



# Problem 3
3. What is the computational cost of the pooling layer? Assume that the input to the
pooling layer is of size c × h × w, the pooling window has a shape of ph × pw with a
padding of (ph, pw) and a stride of (sh, sw).

In [3]:
def pooling_layer_cost(input_size, pooling_window, padding, stride):
    # Unpack input_size, pooling_window, padding, and stride
    c, h, w = input_size
    ph, pw = pooling_window
    ph_pad, pw_pad = padding
    sh, sw = stride

    # Calculate output height and width
    output_height = ((h - ph + 2 * ph_pad) // sh) + 1
    output_width = ((w - pw + 2 * pw_pad) // sw) + 1

    # Compute the total number of output elements
    output_elements = output_height * output_width * c

    # Return the computational cost
    return output_elements

# Example usage
input_size = (3, 32, 32)  # c x h x w
pooling_window = (2, 2)    # ph x pw
padding = (0, 0)           # ph_pad x pw_pad
stride = (2, 2)            # sh x sw

cost = pooling_layer_cost(input_size, pooling_window, padding, stride)
print("Computational cost of the pooling layer:", cost)


Computational cost of the pooling layer: 768
